In [1]:
import pyuvdata
from pyuvdata import UVData
import numpy as np
import glob
import os
from astropy.coordinates import EarthLocation
import astropy.units as u
from pipeline import dsa110_utils

def inspect_hdf5_antenna_positions_v2_4(hdf5_file):
    """Check antenna positions in HDF5 file for pyuvdata 2.4.x"""
    uvd = UVData()
    uvd.read(hdf5_file, file_type='uvh5', run_check=False, read_data=False)
    
    print(f"HDF5 file: {os.path.basename(hdf5_file)}")
    print(f"PyUVData version: {pyuvdata.__version__}")
    
    # In pyuvdata 2.4.x, attributes are directly on the UVData object
    print(f"Telescope name: {getattr(uvd, 'telescope_name', 'Not set')}")
    
    # Check telescope location
    if hasattr(uvd, 'telescope_location') and uvd.telescope_location is not None:
        tel_loc = uvd.telescope_location
        print(f"Telescope location (ITRF): [{tel_loc[0]:.1f}, {tel_loc[1]:.1f}, {tel_loc[2]:.1f}] m")
    else:
        print("❌ No telescope location found!")
    
    # Check antenna positions  
    if hasattr(uvd, 'antenna_positions') and uvd.antenna_positions is not None:
        ant_pos = uvd.antenna_positions
        print(f"Antenna positions shape: {ant_pos.shape}")
        print(f"Antenna position range (relative to telescope):")
        print(f"  X: {ant_pos[:, 0].min():.1f} to {ant_pos[:, 0].max():.1f} m")
        print(f"  Y: {ant_pos[:, 1].min():.1f} to {ant_pos[:, 1].max():.1f} m")
        print(f"  Z: {ant_pos[:, 2].min():.1f} to {ant_pos[:, 2].max():.1f} m")
        
        # Check coordinate system
        max_distance = np.sqrt(np.sum(ant_pos**2, axis=1)).max()
        print(f"Max antenna distance from array center: {max_distance:.1f} m")
        
        if max_distance > 10000:  # > 10 km suggests something's wrong
            print(f"⚠️  Unusually large antenna positions!")
        else:
            print(f"✓ Antenna positions look reasonable")
    else:
        print("❌ No antenna positions found!")
    
    # Check antenna names and numbers
    if hasattr(uvd, 'antenna_names'):
        print(f"Number of antennas: {len(uvd.antenna_names)}")
        print(f"First few antenna names: {uvd.antenna_names[:5]}")
        print(f"Antenna numbers: {uvd.antenna_numbers[:5]}...")
    
    return uvd

def compare_antenna_positions_v2_4(uvd_from_hdf5):
    """Compare HDF5 antenna positions with expected DSA-110 layout - pyuvdata 2.4.x version"""
    
    # Get antenna info from HDF5 (using old API)
    hdf5_ant_pos = uvd_from_hdf5.antenna_positions
    hdf5_ant_names = uvd_from_hdf5.antenna_names
    hdf5_ant_nums = uvd_from_hdf5.antenna_numbers
    
    print("Antenna Position Comparison:")
    print(f"HDF5 antennas: {len(hdf5_ant_names)} antennas")
    print(f"Expected DSA-110 antennas: {len(dsa110_utils.valid_antennas_dsa110)} antennas")
    
    # Check if antenna names match expected pattern
    expected_names = set(dsa110_utils.valid_antenna_names_dsa110)
    hdf5_names = set(hdf5_ant_names)
    
    missing_from_hdf5 = expected_names - hdf5_names
    extra_in_hdf5 = hdf5_names - expected_names
    
    if missing_from_hdf5:
        print(f"Missing from HDF5: {sorted(list(missing_from_hdf5))}")
    if extra_in_hdf5:
        print(f"Extra in HDF5: {sorted(list(extra_in_hdf5))}")
    
    # Check telescope location coordinate system
    if hasattr(uvd_from_hdf5, 'telescope_location') and uvd_from_hdf5.telescope_location is not None:
        # Convert numpy array to EarthLocation for comparison
        tel_loc_array = uvd_from_hdf5.telescope_location  # This is a numpy array [x, y, z] in meters
        telescope_loc_hdf5 = EarthLocation.from_geocentric(
            tel_loc_array[0] * u.m,
            tel_loc_array[1] * u.m, 
            tel_loc_array[2] * u.m
        )
        
        telescope_loc_expected = dsa110_utils.loc_dsa110
        
        print(f"Telescope location in HDF5:")
        print(f"  ITRF XYZ: [{tel_loc_array[0]:.1f}, {tel_loc_array[1]:.1f}, {tel_loc_array[2]:.1f}] m")
        print(f"  Lat/Lon/Alt: {telescope_loc_hdf5.lat.deg:.6f}°, {telescope_loc_hdf5.lon.deg:.6f}°, {telescope_loc_hdf5.height.value:.1f}m")
        
        print(f"Expected DSA-110 location:")
        # Use the proper way to access ITRF coordinates for older astropy
        try:
            # Try newer astropy API first
            expected_itrf = telescope_loc_expected.get_itrs()
            expected_xyz = [expected_itrf.x.value, expected_itrf.y.value, expected_itrf.z.value]
        except:
            # Fall back to older API
            expected_xyz = [telescope_loc_expected.itrs.x.value, telescope_loc_expected.itrs.y.value, telescope_loc_expected.itrs.z.value]
            
        print(f"  ITRF XYZ: [{expected_xyz[0]:.1f}, {expected_xyz[1]:.1f}, {expected_xyz[2]:.1f}] m")
        print(f"  Lat/Lon/Alt: {telescope_loc_expected.lat.deg:.6f}°, {telescope_loc_expected.lon.deg:.6f}°, {telescope_loc_expected.height.value:.1f}m")
        
        # Calculate difference using the arrays directly
        try:
            # Try newer astropy API
            hdf5_itrf = telescope_loc_hdf5.get_itrs()
            hdf5_xyz = [hdf5_itrf.x.value, hdf5_itrf.y.value, hdf5_itrf.z.value]
        except:
            # Fall back to using the original array
            hdf5_xyz = tel_loc_array
            
        location_diff = np.sqrt(np.sum((np.array(hdf5_xyz) - np.array(expected_xyz))**2))
        print(f"Telescope location difference: {location_diff:.3f} m")
        
        if location_diff > 100:  # > 100m difference is significant
            print("⚠️  Large telescope location discrepancy!")
        else:
            print("✓ Telescope locations match reasonably well")
    else:
        print("❌ No telescope location in HDF5 to compare!")

def debug_coordinate_systems_v2_4(uvdata_obj):
    """Debug coordinate system usage in UVW calculation - pyuvdata 2.4.x version"""
    print("Coordinate System Debug (pyuvdata 2.4.x):")
    
    # Check telescope location
    if hasattr(uvdata_obj, 'telescope_location') and uvdata_obj.telescope_location is not None:
        tel_loc = uvdata_obj.telescope_location
        print(f"Telescope location type: {type(tel_loc)}")
        print(f"Telescope location (ITRF XYZ): [{tel_loc[0]:.1f}, {tel_loc[1]:.1f}, {tel_loc[2]:.1f}] m")
        
        # Convert to lat/lon/alt for easier interpretation
        try:
            tel_earthloc = EarthLocation.from_geocentric(
                tel_loc[0] * u.m, tel_loc[1] * u.m, tel_loc[2] * u.m
            )
            print(f"Telescope location (Lat/Lon/Alt): {tel_earthloc.lat.deg:.6f}°, {tel_earthloc.lon.deg:.6f}°, {tel_earthloc.height.value:.1f}m")
        except Exception as e:
            print(f"Could not convert telescope location to Lat/Lon/Alt: {e}")
    else:
        print("❌ No telescope location found!")
    
    # Check antenna positions
    if hasattr(uvdata_obj, 'antenna_positions') and uvdata_obj.antenna_positions is not None:
        ant_pos = uvdata_obj.antenna_positions
        print(f"Antenna positions coordinate system:")
        print(f"  Shape: {ant_pos.shape}")
        print(f"  Reference: relative to telescope location")
        
        max_ant_pos = np.max(np.abs(ant_pos))
        if max_ant_pos < 10000:  # Less than 10 km suggests local coordinates
            print(f"  Frame: Local ENU (max coordinate: {max_ant_pos:.1f} m)")
        else:
            print(f"  Frame: ECEF/ITRF (max coordinate: {max_ant_pos:.1f} m)")
    else:
        print("❌ No antenna positions found!")
    
    # Check UVW coordinates
    if hasattr(uvdata_obj, 'uvw_array') and uvdata_obj.uvw_array is not None:
        max_uvw = np.max(np.abs(uvdata_obj.uvw_array))
        print(f"Max UVW coordinate: {max_uvw:.1f} m")
        
        # Check if antenna positions use consistent coordinates
        if hasattr(uvdata_obj, 'antenna_positions'):
            max_baseline = np.max(np.sqrt(np.sum(uvdata_obj.antenna_positions**2, axis=1)))
            print(f"Max antenna distance from center: {max_baseline:.1f} m")
            
            # Rough consistency check
            if max_uvw > 10 * max_baseline:
                print("⚠️  UVW coordinates seem unusually large compared to antenna positions!")
            else:
                print("✓ UVW and antenna position scales seem consistent")
    else:
        print("❌ No UVW array found!")


    

In [2]:
# Test with your HDF5 files
hdf5_files = glob.glob("/data/incoming/20*_sb00.hdf5")[:2]  # Check first couple

for hdf5_file in hdf5_files:
    print("=" * 80)
    uvd = inspect_hdf5_antenna_positions_v2_4(hdf5_file)
    print()
    compare_antenna_positions_v2_4(uvd)
    print()
    debug_coordinate_systems_v2_4(uvd)
    print("=" * 80)

HDF5 file: 2025-05-13T18:26:14_sb00.hdf5
PyUVData version: 2.4.2
Telescope name: OVRO_MMA
Telescope location (ITRF): [-2409555.9, -4478213.6, 3838767.5] m
Antenna positions shape: (117, 3)
Antenna position range (relative to telescope):
  X: -1125.1 to 1018.5 m
  Y: -269.3 to 1521.3 m
  Z: -304.8 to 1465.5 m
Max antenna distance from array center: 2161.5 m
✓ Antenna positions look reasonable
Number of antennas: 117
First few antenna names: ['1', '2', '3', '4', '5']
Antenna numbers: [0 1 2 3 4]...

Antenna Position Comparison:
HDF5 antennas: 117 antennas
Expected DSA-110 antennas: 96 antennas
Missing from HDF5: ['pad1', 'pad100', 'pad101', 'pad102', 'pad103', 'pad104', 'pad105', 'pad106', 'pad107', 'pad108', 'pad109', 'pad11', 'pad110', 'pad111', 'pad112', 'pad113', 'pad114', 'pad115', 'pad116', 'pad12', 'pad13', 'pad14', 'pad15', 'pad16', 'pad17', 'pad18', 'pad19', 'pad2', 'pad20', 'pad24', 'pad25', 'pad26', 'pad27', 'pad28', 'pad29', 'pad3', 'pad30', 'pad31', 'pad32', 'pad33', 'pad34'

Telescope OVRO_MMA is not in known_telescopes.


In [31]:
# Test with your HDF5 files
hdf5_files = glob.glob("/data/incoming/20*_sb00.hdf5")[:2]  # Check first couple
for hdf5_file in hdf5_files:
    uvd = inspect_hdf5_antenna_positions_v2_4(hdf5_file)
    compare_antenna_positions(uvd)
    print("-" * 60)

HDF5 file: 2025-05-13T18:26:14_sb00.hdf5
PyUVData version: 2.4.2
Telescope name: OVRO_MMA
Telescope location (ITRF): [-2409555.9, -4478213.6, 3838767.5] m
Antenna positions shape: (117, 3)
Antenna position range (relative to telescope):
  X: -1125.1 to 1018.5 m
  Y: -269.3 to 1521.3 m
  Z: -304.8 to 1465.5 m
Max antenna distance from array center: 2161.5 m
✓ Antenna positions look reasonable
Number of antennas: 117
First few antenna names: ['1', '2', '3', '4', '5']
Antenna numbers: [0 1 2 3 4]...
Antenna Position Comparison:
HDF5 antennas: 117 antennas
Expected DSA-110 antennas: 96 antennas
Missing from HDF5: ['pad1', 'pad100', 'pad101', 'pad102', 'pad103', 'pad104', 'pad105', 'pad106', 'pad107', 'pad108', 'pad109', 'pad11', 'pad110', 'pad111', 'pad112', 'pad113', 'pad114', 'pad115', 'pad116', 'pad12', 'pad13', 'pad14', 'pad15', 'pad16', 'pad17', 'pad18', 'pad19', 'pad2', 'pad20', 'pad24', 'pad25', 'pad26', 'pad27', 'pad28', 'pad29', 'pad3', 'pad30', 'pad31', 'pad32', 'pad33', 'pad34',

Telescope OVRO_MMA is not in known_telescopes.


AttributeError: 'numpy.ndarray' object has no attribute 'itrs'

In [25]:
# Test with your HDF5 files
hdf5_files = glob.glob("/data/incoming/20*_sb00.hdf5")[:2]  # Check first couple
for hdf5_file in hdf5_files:
    compare_antenna_positions(hdf5_file)
    print("-" * 60)

AttributeError: 'str' object has no attribute 'antenna_positions'